In [14]:
## Este script combina los archivos CSV de data sin tener en cuenta los de "Happines.."
## Los indicadores para cada año se van guardando fila por fila
import os
import pandas as pd
import json

# Obtener la ruta del directorio del notebook actual
current_dir = os.getcwd()

# Construir la ruta relativa a la carpeta 'datos'
folder_path = os.path.join(current_dir, '../data')

# Leer el archivo JSON con la codificación correcta
with open('data_country.json', 'r', encoding='utf-8') as file:
    data_country = json.load(file)

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
for country, info in data_country.items():
    for alt_name in info.get('AlternateNames', []):
        alternate_names_to_official[alt_name] = country

# Obtener una lista de todos los archivos CSV en la carpeta, excluyendo los que contienen "Happiness"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and 'Happiness' not in f]

# Inicializar un DataFrame vacío para almacenar todos los datos transpuestos
all_transposed_data = pd.DataFrame()

# Leer y procesar cada archivo CSV
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    try:
        # Leer el archivo CSV con el delimitador adecuado
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Encontrar el índice de la fila "Country Name"
        start_index = None
        for i, line in enumerate(lines):
            if line.startswith('"Country Name"'):
                start_index = i
                break
        
        if start_index is None:
            print(f'No se encontró la fila "Country Name" en el archivo {csv_file}')
            continue
                
        # Leer el archivo CSV desde la fila "Country Name"
        data = pd.read_csv(file_path, encoding='utf-8', delimiter=',', quotechar='"', on_bad_lines='skip', skiprows=start_index)
        
        # Reemplazar nombres alternativos por nombres oficiales
        data['Country Name'] = data['Country Name'].apply(lambda x: alternate_names_to_official.get(x, x))  
        
        # Cambiar la información de la columna "Indicator Name" por el nombre de la carpeta
        data['Indicator Name'] = os.path.basename(file_path)
        
        # Eliminar las columnas "Country Code" y "Indicator Code" si existen
        if 'Country Code' in data.columns:
            data = data.drop(columns=['Country Code'])
        if 'Indicator Code' in data.columns:
            data = data.drop(columns=['Indicator Code'])
        
        # Validar que todos los países de la primera columna se encuentren en el archivo JSON
        valid_countries = data['Country Name'].isin(data_country.keys())
        data_cleaned = data[valid_countries]
        
        # Eliminar las filas vacías del DataFrame original
        filas_vacias = data_cleaned[data_cleaned.iloc[:, 0].isnull() | (data_cleaned.iloc[:, 0] == '')]
        data_cleaned = data_cleaned.drop(filas_vacias.index)
        
        # Eliminar columnas de años que no se necesitan
        columns_to_keep = list(data_cleaned.columns[:4]) + list(data_cleaned.columns[-10:])
        data_cleaned = data_cleaned[columns_to_keep]

        # Crear columnas adicionales para Región y Continente
        data_cleaned['Region'] = data_cleaned['Country Name'].apply(lambda x: data_country.get(x, {}).get('Region', 'Región Desconocida'))
        data_cleaned['Continent'] = data_cleaned['Country Name'].apply(lambda x: data_country.get(x, {}).get('Continent', 'Continente Desconocido'))
        
        # Crear una lista para almacenar las filas transpuestas
        transposed_data = []
        
        # Se define desde que columna lee los años
        years = data_cleaned.columns[4:]

        # Iterar sobre cada fila del DataFrame original
        for index, row in data_cleaned.iterrows():
            country_name = row['Country Name']
            indicator_name = row['Indicator Name']
            region = row['Region']
            continent = row['Continent']
    
            # Iterar sobre las columnas de años
            for year in years[:-2]:  # Excluir 'Unnamed: 68' y 'Most_Recent_Value'
                value = row[year]
                if not pd.isna(value):
                    transposed_data.append([country_name, indicator_name, year, value, region, continent])

        # Crear un nuevo DataFrame con los datos transpuestos
        transposed_df = pd.DataFrame(transposed_data, columns=['Country Name', 'Indicator Name', 'Year', 'Value', 'Region', 'Continent'])

        # Concatenar los datos transpuestos al DataFrame principal
        all_transposed_data = pd.concat([all_transposed_data, transposed_df], ignore_index=True)
        
    except pd.errors.ParserError as e:
        print(f'Error al procesar el archivo {csv_file}: {e}')

# Ordenar el DataFrame combinado por el nombre de la columna "Country Name"
all_transposed_data.sort_values(by=['Country Name', 'Indicator Name'], inplace=False)

# Guardar el dataframe consolidado en un nuevo archivo CSV
all_transposed_data.to_csv('data_total_country_xfila.csv', index=False)

print("Consolidación completa. Los datos consolidados se han guardado en data_total_country_xfila.csv.")

Consolidación completa. Los datos consolidados se han guardado en data_total_country_xfila.csv.


In [13]:
## Este script combina los archivos CSV de data sin tener en cuenta los de "Happines.."
## Los indicadores estaran en las columnas junto con el año
import os
import pandas as pd
import json

# Obtener la ruta del directorio del notebook actual
current_dir = os.getcwd()

# Construir la ruta relativa a la carpeta 'datos'
folder_path = os.path.join(current_dir, '../data')

# Leer el archivo JSON con la codificación correcta
with open('data_country.json', 'r', encoding='utf-8') as file:
    data_country = json.load(file)

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
for country, info in data_country.items():
    for alt_name in info.get('AlternateNames', []):
        alternate_names_to_official[alt_name] = country

# Obtener una lista de todos los archivos CSV en la carpeta, excluyendo los que contienen "Happiness"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and 'Happiness' not in f]

# Inicializar un DataFrame vacío para almacenar todos los datos
consolidated_data = pd.DataFrame()
    
# Leer y procesar cada archivo CSV
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    file_name = os.path.splitext(csv_file)[0] # Devuelve el nombre del archivo sin la extensión
        
    try:
        # Leer el archivo CSV con el delimitador adecuado
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Encontrar el índice de la fila "Country Name"
        start_index = None
        for i, line in enumerate(lines):
            if line.startswith('"Country Name"'):
                start_index = i
                break
        
        if start_index is None:
            print(f'No se encontró la fila "Country Name" en el archivo {csv_file}')
            continue
                
        # Leer el archivo CSV desde la fila "Country Name"
        data = pd.read_csv(file_path, encoding='utf-8', delimiter=',', quotechar='"', on_bad_lines='skip', skiprows=start_index)
        
        # Reemplazar nombres alternativos por nombres oficiales
        data['Country Name'] = data['Country Name'].apply(lambda x: alternate_names_to_official.get(x, x))  
        
        # Validar que todos los países de la primera columna se encuentren en el archivo JSON
        valid_countries = data['Country Name'].isin(data_country.keys())
        data_cleaned = data[valid_countries]
        
        # Eliminar las filas vacías del DataFrame original
        filas_vacias = data_cleaned[data_cleaned.iloc[:, 0].isnull() | (data_cleaned.iloc[:, 0] == '')]
        data_cleaned = data_cleaned.drop(filas_vacias.index)
        
        # Eliminar columnas de años que no se necesitan
        columns_to_keep = list(data_cleaned.columns[:4]) + list(data_cleaned.columns[-10:])
        data_cleaned = data_cleaned[columns_to_keep]
            
        # Cambiar la información de la columna "Indicator Name" por el nombre del archivo CSV más el año
        for year in data_cleaned.columns[4:]:
            new_column_name = f"{file_name}_{year}"
            data_cleaned.rename(columns={year: new_column_name}, inplace=True)
            data_cleaned[new_column_name] = data_cleaned[new_column_name].round(3)    
        
        # Crear columnas adicionales para Región y Continente si consolidated_data está vacío
        if consolidated_data.empty:
            data_cleaned['Region'] = data_cleaned['Country Name'].apply(lambda x: data_country.get(x, {}).get('Region', 'Región Desconocida'))
            data_cleaned['Continent'] = data_cleaned['Country Name'].apply(lambda x: data_country.get(x, {}).get('Continent', 'Continente Desconocido'))
        
            # Reordenar las columnas para que 'Region' y 'Continent' estén después de 'Country Name'
            columns_order = ['Country Name', 'Region', 'Continent'] + [col for col in data_cleaned.columns if col not in ['Country Name', 'Region', 'Continent']]
            data_cleaned = data_cleaned[columns_order]
        
        # Eliminar las columnas "Country Code", "Indicator Code" y "Indicator Name" si existen
        columns_to_drop = ['Country Code', 'Indicator Code', 'Indicator Name']
        for column in columns_to_drop:
            if column in data_cleaned.columns:
                data_cleaned = data_cleaned.drop(columns=[column])

        # Eliminar las columnas con datos vacíos
        data_cleaned = data_cleaned.dropna(axis=1, how='all')    
                
        # Si es el primer archivo, inicializar el DataFrame consolidado
        if consolidated_data.empty:
            consolidated_data = data_cleaned
        else:
            # Combinar el DataFrame consolidado con el nuevo DataFrame
            consolidated_data = pd.merge(consolidated_data, data_cleaned, on=['Country Name'], how='outer')
            
            # Añadir Región y Continente para los nuevos países
            for country in data_cleaned['Country Name']:
                if country not in consolidated_data['Country Name'].values:
                    region = data_country.get(country, {}).get('Region', 'Región Desconocida')
                    continent = data_country.get(country, {}).get('Continent', 'Continente Desconocido')
                    consolidated_data.loc[consolidated_data['Country Name'] == country, 'Region'] = region
                    consolidated_data.loc[consolidated_data['Country Name'] == country, 'Continent'] = continent

    except pd.errors.ParserError as e:
        print(f'Error al procesar el archivo {csv_file}: {e}')

# Ordenar el DataFrame combinado por el nombre de la columna "Country Name"
consolidated_data.sort_values(by=['Country Name'], inplace=False)

# Guardar el dataframe consolidado en un nuevo archivo CSV
consolidated_data.to_csv('data_total_country_xcolumna.csv', index=False)

print("Consolidación completa. Los datos consolidados se han guardado en data_total_country_xcolumna.csv.")

Consolidación completa. Los datos consolidados se han guardado en data_total_country_xcolumna.csv.
